<a href="https://colab.research.google.com/github/sergey-khvan/phrase-suggestions/blob/main/suggestions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=6045e716865272ea89044455c98a21377acefe9780f1713d08d9b853db690b9a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [85]:
from sentence_transformers import SentenceTransformer
import numpy as np
import re
import csv

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [55]:
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

In [56]:
with open('task/text.txt', 'r') as file:
    text = file.read()
with open('task/terms.csv', 'r') as file:
    # Create a CSV reader
    csv_reader = csv.reader(file)

    # Extract phrases into a list
    phrase_list = []
    for row in csv_reader:
        phrase = row[0]
        phrase_list.append(phrase)

In [71]:
import re
delimiter = '.,'

# Split the text using the custom delimiter
result = re.split('[' + delimiter + ']', text)

# Remove any empty strings from the result
sentences = [item.strip().lower() for item in result if item.strip()]

In [73]:
phrase_list = [phrase.lower() for phrase in phrase_list]

In [74]:
sentence_emb = model.encode(sentences, convert_to_tensor=True)

In [75]:
phrase_emb = model.encode(phrase_list, convert_to_tensor=True)

In [79]:
similarities= []
for i, sentence in enumerate(sentence_emb):
    max_similarity = -1
    most_similar_phrase = None
    for j, phrase in enumerate(phrase_emb):
        similarity = cosine_similarity(sentence, phrase)
        if similarity > 0.35 and similarity > max_similarity:
            max_similarity = similarity
            most_similar_phrase = phrase_list[j]
    if most_similar_phrase is not None:
        similarities.append((sentences[i], most_similar_phrase, max_similarity))


In [80]:
similarities

[('we discussed a variety of issues affecting our department',
  'conduct an analysis',
  0.40268415),
 ('a pleasant backdrop to our serious discussions',
  'facilitate collaboration',
  0.3612662),
 ('we came to the consensus that we need to do better in terms of performance',
  'optimal performance',
  0.5739656),
 ("it's important to make good use of what we have at our disposal",
  'utilise resources',
  0.44738084),
 ('we should aim to be more efficient and look for ways to be more creative in our daily tasks',
  'enhance productivity',
  0.5526855),
 ('growth is essential for our future', 'drive growth', 0.52710027),
 ('but equally important is building strong relationships with our team members',
  'facilitate collaboration',
  0.49486223)]